In [17]:
import torch
from tqdm import tqdm
from model import seedformer_dim128

from torch.utils.data import Dataset,DataLoader
import open3d as o3d
import os
from pytorch3d import loss 

### Dataset and Dataloader

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [11]:
class RacingDataset(Dataset):
    def __init__(self,root_dir):
        self.root_dir = root_dir
        self.file_list = os.listdir(root_dir)
        self.filter_file_list = self.filter_list()

    def __len__(self):
        return len(self.filter_file_list)

    def __getitem__(self,index):
        pcd_path = os.path.join(self.root_dir,self.filter_file_list[index])
        pcd = o3d.io.read_point_cloud(pcd_path)

        points = torch.tensor(pcd.points, dtype=torch.float32)

        return points,pcd_path

    def filter_list(self):
        '''
        Filter the inputs so that only pcds with more than 50 points are included in the training
        :return:
        '''
        filtered_list=[]
        for filename in self.file_list:
            pcd = o3d.io.read_point_cloud(os.path.join(self.root_dir,filename))
            points = torch.tensor(pcd.points, dtype=torch.float32)
            if len(points)>=50:
                filtered_list.append(filename)
        return filtered_list


In [50]:
dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real")
dataloader = DataLoader(dataset,batch_size=1,shuffle=True,num_workers=4)
print(len(dataloader))
print(dataloader.dataset)

4097


In [60]:
torch.manual_seed(42)
model=seedformer_dim128(up_factors=[1, 2, 2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
epochs=5
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
for epoch in range(epochs):
    #loss = loss.chamfer_distance()
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data  # Note: We have dummy labels here
        inputs = inputs.to(device)  # Move data to GPU if available
        optimizer.zero_grad()
        #print(inputs)
        outputs = model(inputs)        # Calculate Chamfer Distance loss
        #outputs = torch.stack([output for output in outputs if output.shape[0] == inputs.shape[1]], dim=0)

        print("outoutputs",len(outputs))
        loss, _ = loss.chamfer_distance(inputs, outputs)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')
        

forward - npoints: None, self.nsample: 4
sample xyz.shape  torch.Size([1, 3, 259])
sample npoints: -   40
forward - npoints: None, self.nsample: 8
sample xyz.shape  torch.Size([1, 3, 40])
sample npoints: -   20
forward - npoints: None, self.nsample: None
sample xyz.shape  torch.Size([1, 3, 20])
sample npoints: -   None
outoutputs 4


ValueError: The input pointclouds should be either Pointclouds objects or torch.Tensor of shape (minibatch, num_points, 3).